## 基于 CRF 的命名实体识别

In [7]:
import pandas as pd
import numpy as np

data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [8]:
data.tail(10)

Sentence #       Word  POS    Tag
1048565              NaN     impact   NN      O
1048566              NaN          .    .      O
1048567  Sentence: 47959     Indian   JJ  B-gpe
1048568              NaN     forces  NNS      O
1048569              NaN       said  VBD      O
1048570              NaN       they  PRP      O
1048571              NaN  responded  VBD      O
1048572              NaN         to   TO      O
1048573              NaN        the   DT      O
1048574              NaN     attack   NN      O

In [9]:
data = data.fillna(method="ffill") # 向下填充

In [10]:
data.tail(10)

Sentence #       Word  POS    Tag
1048565  Sentence: 47958     impact   NN      O
1048566  Sentence: 47958          .    .      O
1048567  Sentence: 47959     Indian   JJ  B-gpe
1048568  Sentence: 47959     forces  NNS      O
1048569  Sentence: 47959       said  VBD      O
1048570  Sentence: 47959       they  PRP      O
1048571  Sentence: 47959  responded  VBD      O
1048572  Sentence: 47959         to   TO      O
1048573  Sentence: 47959        the   DT      O
1048574  Sentence: 47959     attack   NN      O

In [12]:
words = list(set(data["Word"].values))
n_words = len(words)
n_words

35178

所以我们有47959个句子，包含35178个不同的单词。

定义一个句子获取的类：

In [17]:
class SentenceGetter:
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values,
                                                           s["POS"].values,
                                                           s["Tag"].values)]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func) # 按照句子编号进行分组
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [18]:
getter = SentenceGetter(data)
sent = getter.get_next()
sent

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

获取所有的句子：

In [20]:
sentences = getter.sentences
len(sentences)

47959

特征定义：

In [21]:
def word2features(sent, i):
    # i 表示单词在句子中的 index
    word = sent[i][0] # 单词本身
    postag = sent[i][1] # 词性

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),# 小写后的单词
        'word[-3:]': word[-3:], # 后缀
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(), # 是否为大写
        'word.istitle()': word.istitle(), # 单词是否首字母为大写，其它字母为小写
        'word.isdigit()': word.isdigit(), # 是否为数字
        'postag': postag, # 词性
        'postag[:2]': postag[:2], # 前缀
    }
    if i > 0: # 如果单词不在句首，获取其前一单词的相关特征
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1: # 如果单词不在句尾，获取其后一单词的相关特征
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

In [22]:
# 句子转化为特征
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

# 取句子的标签
def sent2labels(sent):
    return [label for token, postag, label in sent]

# 句子转化为 token
def sent2tokens(sent):
    return [token for token, postag, label in sent]

获取输入输出：

In [23]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

初始化 CRF：

In [24]:
from sklearn_crfsuite import CRF
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)


训练 CRF：

In [27]:
import sklearn
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

d:\anaconda\envs\python36\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [28]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

d:\anaconda\envs\python36\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

       B-art       0.37      0.11      0.17       402
       B-eve       0.52      0.35      0.42       308
       B-geo       0.85      0.90      0.88     37644
       B-gpe       0.97      0.94      0.95     15870
       B-nat       0.66      0.37      0.47       201
       B-org       0.78      0.72      0.75     20143
       B-per       0.84      0.81      0.82     16990
       B-tim       0.93      0.88      0.90     20333
       I-art       0.11      0.03      0.04       297
       I-eve       0.34      0.21      0.26       253
       I-geo       0.82      0.79      0.80      7414
       I-gpe       0.92      0.55      0.69       198
       I-nat       0.61      0.27      0.38        51
       I-org       0.81      0.79      0.80     16784
       I-per       0.84      0.89      0.87     17251
       I-tim       0.83      0.76      0.80      6528
           O       0.99      0.99      0.99    887908

    accuracy              

In [31]:
crf.fit(X, y)

d:\anaconda\envs\python36\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [32]:
import eli5
eli5.show_weights(crf, top=30)

加大正则化力度：

In [33]:
crf = CRF(algorithm='lbfgs',
        c1=10,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=False)
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

d:\anaconda\envs\python36\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
d:\anaconda\envs\python36\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       402
       B-eve       0.80      0.27      0.40       308
       B-geo       0.82      0.90      0.86     37644
       B-gpe       0.95      0.92      0.94     15870
       B-nat       0.69      0.09      0.16       201
       B-org       0.78      0.67      0.72     20143
       B-per       0.80      0.76      0.78     16990
       B-tim       0.93      0.83      0.88     20333
       I-art       0.00      0.00      0.00       297
       I-eve       0.64      0.12      0.20       253
       I-geo       0.81      0.73      0.77      7414
       I-gpe       0.93      0.37      0.53       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.75      0.76      0.75     16784
       I-per       0.80      0.90      0.85     17251
       I-tim       0.84      0.67      0.74      6528
           O       0.99      0.99      0.99    887908

    accuracy              

In [34]:
crf.fit(X, y)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=10, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [35]:
eli5.show_weights(crf, top=30)